<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex13notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex13notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo13.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## クラスター分析／クラスタリング
---



In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

---
### クラスター分析／クラスタリングとは

判別分析は，データを分類するための手法でした．今回説明する **クラスター分析** (cluster analysis)／ **クラスタリング** (clustering)（注） も同様の手法です．ただし，判別分析ではデータの分類の仕方があらかじめ決まっている（データの中にクラスを表す変数が存在する）問題が対象ですが，クラスタリングでは，分類の仕方が決まっていない問題を扱います．


例えば，イヌ／ネコ／カピバラ3種の写真を集めたデータがあったとしましょう．判別分析では，写真に分類の仕方の正解（3種のどれか）を表す情報が付け加えられたデータを扱います．このデータを用いてパラメータを推定（分類の仕方を学習）し，そのパラメータを用いて，未知の写真に写っているのが3種のどれかを予測します．

一方，クラスタリングでは，分け方を知らないデータを対象として，いくつのグループに分けられるか，どのように分けられるかを分析によって明らかにしようとします．分類の仕方の正解はありませんので，上の例のような写真のデータを対象としても3種をうまく分けるとは限りませんし，3つではなく2つや5つに分けるかもしれません．地球の動物を知らない宇宙人が写真を見て動物を分類しようとしている，と考えたらわかりやすいでしょうか．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「クラスター分析」と「クラスタリング」は同じものを指す語です．統計学方面では前者が，データサイエンスや機械学習方面では後者が使われることが多いようです．以降では「クラスタリング」の方を使います．
</span>

クラスタリングは，上述のようにデータをいくつかの「塊」に分ける手法です．その「塊」のことを **クラスタ** (cluster) といいます．クラスタリングを行うことで，データの構造を把握（いくつのクラスタに分けられるのかを知る，どのデータとどのデータが似ているのかを知る，等）したり，大量のデータを要約（クラスタごとにデータの性質を記述する，等）したりすることができます．

例1: 学生たちの20科目の成績を表すデータ（$D=20$）をクラスタリング．

クラスタリングによって，例えば，3つのクラスタに分けられることが分かったならば...．クラスタごとに成績の傾向が異なるので，学習指導の仕方をそれに応じて変えるようなことが可能となる．


例2: 遺伝子データをクラスタリング

様々な生物種の遺伝子データ（DNA配列など）をクラスタリングして解析できたならば...．
生物種の間の類縁関係を推定（種Aと種Bは同じクラスタに分類されるので，同じ種から進化した可能性が高いと考える，等）したりできるかもしれない．
ウイルスの変異を分析したりも...．

例3: ネットショッピングの買い物傾向を表すデータをクラスタリング．

ネットショッピングサイトの経営者が，顧客がどんな商品を購入したかのデータをクラスタリングして，顧客を買い物傾向によってグループ分けできたならば...．
顧客の傾向に応じてサイトに表示させる広告やおすすめ商品の表示を変えることができる（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: ここでは直感的な例としてこのような問題設定をあげましたが，実際には，この問題は,クラスタリングとは少し設定が異なるところがあり，「協調フィルタリング」という問題として扱われています．
</span>

---
### 階層型クラスタリングと非階層型クラスタリング





クラスタリングの手法は，**階層型クラスタリング** と **非階層型クラスタリング** に大別できます．

**階層型クラスタリング** (hierarchical clustering): 「クラスタAとクラスタBをあわせたものがクラスタPで，クラスタPとクラスタQをあわせたものがクラスタR」というように，階層的になったクラスタを作るクラスタリング手法．
さらに以下の二つに分けられる：
- **凝集型クラスタリング** (agglomerative clustering): 個々のデータをそれぞれ一つのクラスタとした状態から始めて，だんだんクラスタをくっつけていく
- **分割型（分枝型）クラスタリング** (divisive clustering) : 全てのデータを一つのクラスタとした状態から始めて，それを分割していく

**非階層型クラスタリング** (non-hierarchical clustering): クラスタ同士に上記のような階層構造をつくらないクラスタリング手法．**$K$-平均法** ($K$-means method) が代表例．様々な手法があり，性質やアルゴリズムによって細分類できるが，この授業では省略する（$K$-平均法は「機械学習I/II」で説明します）．





凝集型クラスタリングの一例を以下の図に示します．詳しい説明はまた後でします．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/hierarchy-anim.gif">

---
### 凝集型クラスタリング

凝集型クラスタリングについて説明します．この授業では，分割型クラスタリングについては説明を省略します．

#### 凝集型クラスタリングの手順とデンドログラム

凝集型クラスタリングの手順は，大まかには次のようになります．

(0) 個々の標本をそれぞれクラスタとする．

(1) 現在のクラスタたちの間の距離を求め，最も距離の小さかった二つのクラスタを合併して一つのクラスタにする．

(2) クラスタが一つだけになっていたら終了．さもなくば(1) へ戻る．

凝集型クラスタリングでは，「クラスタとクラスタの間の距離」をどのように測るかが重要なポイントで，様々な方法が考えられています（あとで説明します）．

階層型クラスタリングでは，クラスタリングを行って得られるクラスタの階層構造を調べて，データの特徴を分析することになります．クラスタの階層構造は，**デンドログラム**（**樹形図**，dendrogram）という図に表すことがよくあります．

以下の図の右は，左のデータに凝集型クラスタリングを適用して得られたクラスタの構造を表すデンドログラムです（↑に載せてるものの再掲）．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hierarchy-anim.gif">

このデンドログラムの縦軸は，クラスタ間の距離を表しています．1番と2番の標本に注目すると，下の方では別々の縦線が描かれていたものが，少し上にたどると一つに合わさっています．これは，この地点でこの二つの標本（のそれぞれのみから成るクラスタ）が合わさって一つのクラスタになることを意味しています．このときの縦軸の値は，両者の間の距離を表しています．

そのままデンドログラムを上に登っていくと，次は「1番と2番の合わさったクラスタ」と3番の標本が一つのクラスタになることも分かりますね．以下，同様にして，だんだんとクラスタが合併していく様子を見ることができます．

階層型クラスタリングでは，このような樹形図を見て，データを最終的にいくつのクラスタに分けるかを判断することができます．この例では，距離の基準（しきい値）を $3$ とすれば，「0-4番を含むクラスタ」と「5-9番を含むクラスタ」の2つに分けられますし，$2$ とすれば，「0-3番を含むクラスタ」，「4番のみのクラスタ」，「5,6,7,9番を含むクラスタ」，「8番のみのクラスタ」の4つに分けられます．
このように，結果を見てクラスタの分け方を選べるのが階層型クラスタリングの特徴です．

#### クラスタ間の距離の測り方

先にも書いたように，階層型クラスタリングにおいては，「クラスタとクラスタの間の距離」をどのように測るかが重要なポイントとなります．
データの性質に応じて様々なものが考えられますが，
量的な特徴量のみから成るデータの場合，

1. 標本間の距離を定義する
1. それに基づいてクラスタ間の距離を定義する

とします（注）．標本間の距離とクラスタ間の距離の定義が異なることに注意．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 詳しくは説明しませんが，質的な特徴量に対しても適切に距離を定義してやることでクラスタリングが可能となります．
</span>



標本間の距離としては，おなじみのユークリッド距離などがよく用いられます．二つのデータを
$$
\pmb{x} = (x_1, x_2, \ldots, x_D),\quad \pmb{y} = (y_1, y_2, \ldots, y_D)
$$
として，これらの間の距離を $d(\pmb{x}, \pmb{y})$ と表すとき，ユークリッド距離ならば
$$
d(\pmb{x}, \pmb{y}) = \sqrt{ \sum_{d=1}^{D}(x_d - y_d)^2 }
$$
ですね．他にも，次式で表される「マンハッタン距離(Manhattan distance)」（注）なども用いられます．
$$
d(\pmb{x}, \pmb{y}) = \sum_{d=1}^{D}|x_d - y_d|
$$

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 余談ですが，ユークリッド距離やマンハッタン距離を一般化したものとして，「ミンコフスキー距離」というものがあります．定義は
$$
d(\mathbf{x}, \mathbf{y}) = \left( \sum_{d=1}^{D}|x_d - y_d|^{p} \right)^{\frac{1}{p}}
$$
です．$p = 2$ ならユークリッド距離，$p=1$ ならマンハッタン距離です．総称して，「$L_p$距離」という言い方をすることもあります．ユークリッド距離は $L_2$ 距離，マンハッタン距離は $L_1$ 距離．
</span>


クラスタとクラスタの間の距離の測り方には，様々なものがあります．
代表的なものをいくつか挙げておきます（個別の方法を覚えることは重要ではありません．実際に使うことになったときにあらためて調べればよいでしょう）．

**単リンク法**（single linkage method）

> クラスタ$C_1$とクラスタ$C_2$ の間の距離 $d(C_1, C_2)$を次式で測る方法：
$$
d(C_1, C_2) = \min_{\mathbf{x}_1\in C_1, \pmb{x}_2\in C_2} d(\pmb{x}_1, \pmb{x}_2)
$$
つまり，2つのクラスタからひとつずつ任意にデータを選んで距離を測ったときに，その最小値をクラスタ間の距離とする．

**完全リンク法**（complete linkage method）

> クラスタ$C_1$とクラスタ$C_2$ の間の距離 $d(C_1, C_2)$を次式で測る方法：
$$
d(C_1, C_2) = \max_{\pmb{x}_1\in C_1, \pmb{x}_2\in C_2} d(\pmb{x}_1, \pmb{x}_2)
$$
単リンク法と逆で，2つのクラスタからひとつずつ任意に点を選んで距離を測ったときに，その最大値をクラスタ間の距離とする．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/single_complete_linkage.png">

上の図にこれらの方法のイメージを示します．単リンク法の場合，クラスタ $C$ とクラスタ $A$ との間の距離 $d(C,A)$ およびクラスタ $C$ とクラスタ $B$ との間の距離 $d(C,B)$ は，それぞれ左図の太線で示された2点間の距離で与えられます．一方，完全リンク法の場合は右図のようになります．この例の場合，クラスタ $C$ との距離がより小さいのは，単リンク法ではクラスタ $B$ ，完全リンク法ではクラスタ $A$ ということになります．

**ウォード法** (Ward's method)

> クラスタ$C_1$とクラスタ$C_2$ の間の距離 $d(C_1, C_2)$を次式で測る方法：
$$
d(C_1, C_2) = e^2(C_1 \cup C_2) - (e^2(C_1) + e^2(C_2))
$$
ただし，$e^2(C)$ は，クラスタ $C$ に所属するすべての点と，それらの重心 $\widebar{\pmb{x}}$ との間の距離の二乗和，すなわち
$$
e^2(C) = \sum_{\pmb{x}\in C}d^2(\pmb{x}, \bar{\pmb{x}})
$$
この値は，$C$内の点の「散らばりの大きさ」を表すとみなせる．したがって，$d(C_1, C_2)$ は，「クラスタ $C_1$ と $C_2$ を合併して一つのクラスタにしたときに散らばりの大きさがどれだけ増加するか」を測っていることになる．

ウォード法のイメージを下図に示します．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Ward.png">

単リンク法や完全リンク法などに比べてウォード法の方が「よい」（注）クラスタリング結果が得られると考えられており，一般的にはウォード法がよく用いられます．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「よくない」結果の例としては，あるクラスタに標本が1つずつ追加されてクラスタが大きくなっていくような場合があげられます．そのような結果になると，デンドログラムをどこで切ってもクラスタとばらばらの点群に分けられてしまい，うまく「塊」を見いだせません．
</span>


よく用いられるものはこれ以外にもありますが，説明は省略します．
興味があれば，以下のリンクをたどってみてね．
- Python の科学技術計算ライブラリ SciPy https://scipy.org/
    - の中の階層型クラスタリングパッケージ [scipy.cluster.hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html)
        - の中のクラスタリングの関数 [scipy.cluster.hierarchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage)． 様々なクラスタ間距離の測り方が使えるようになっていて，その説明もあります．
- Python の機械学習ライブラリ scikit-learn https://scikit-learn.org/
    - の階層型クラスタリングに関するドキュメント https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering
    - 凝集型クラスタリングのクラス [sklearn.cluster.AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

#### 実験: 2次元データのクラスタリング

2次元のデータを実際にクラスタリングしてみましょう．

In [ ]:
# 実験用データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/data4kmeans.csv')
X = df[['x1', 'x2']].to_numpy()
print(X[:5, :], X.shape)

In [ ]:
# 散布図
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1])
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
plt.show()

In [ ]:
#@title 2次元データの階層型クラスタリング
#@markdown `method` でクラスタ間距離の計算法を指定．
#@markdown  - `single` = 単リンク法
#@markdown  - `complete`= 完全リンク法
#@markdown  - `ward` = ウォード法
#@markdown
#@markdown `threshold` でクラスタ間距離のしきい値を指定

method = 'ward' #@param ['single', 'complete', 'ward']
threshold = 10 #@param [0.7, 1.0, 5.0, 7.0, 10.0] {type: "raw"}

# 標本間の距離をユークリッド距離で測り，クラスタ間距離を method で
# 指定した方法で測るクラスタリング
link = hierarchy.linkage(X, method=method,
                         metric='euclidean')

# threshold の値を距離のしきい値としてデータをクラスタ分け
label = hierarchy.fcluster(link, t=threshold, criterion='distance')
ncluster = np.max(label)

# グラフ描画の準備
fig = plt.figure(facecolor="white", figsize=(13, 6))

# デンドログラムを描く
ax0 = fig.add_subplot(121)
hierarchy.dendrogram(link, color_threshold=threshold, distance_sort='descending', show_leaf_counts=True, ax=ax0)
ax0.axhline(threshold, color='red', linestyle='--')

# クラスタ分けした学習データを描く
ax1 = fig.add_subplot(122)
for k in range(ncluster):
    XX = X[label==k+1, :]
    ax1.scatter(XX[:, 0], XX[:, 1])
ax1.text(1.2, 4, f'#cluter = {ncluster}', size=18)
ax1.set_xlim(-5, 5)
ax1.set_ylim(-5, 5)
ax1.set_aspect('equal')

plt.show()

上記のグラフでは，デンドログラムの枝が塗り分けられていますが，その色は右の図のクラスタの塗り分けの色とは一致していません．注意してください．

#### ★★ やってみよう ★★

1. 条件をいろいろ変えてセルを実行してみよう．
1. 次の条件のときにデータがいくつのクラスタに分けられたか，紙にメモしておこう
- 単リンク法でクラスタ間距離のしきい値が 1 のとき
- ウォード法でクラスタ間距離のしきい値が 10 のとき